# Puts ALL WISE Astrometry reference catalogues into GAIA reference frame

<img src=https://avatars1.githubusercontent.com/u/7880370?s=200&v=4>

The WISE catalogues were produced by ../dmu16_allwise/make_wise_samples_for_stacking.csh

In the catalogue, we keep:

- The position;
- The chi^2

This astrometric correction is adapted from master list code  (dmu1_ml_XMM-LSS/1.8_SERVS.ipynb) written by Yannick Rohlly and Raphael Shirley


In [1]:
field="AKARI-SEP"
field="AKARI-NEP"
field='Herschel-Stripe-82' 

field='HDF-N' 
field='XMM-LSS' 
field='AKARI-NEP' 
field='AKARI-SEP' 
field='Bootes' 
field='CDFS-SWIRE' 
field='COSMOS' 
field='EGS' 
field='ELAIS-N1' 
field='ELAIS-N2' 
field='ELAIS-S1' 
field='GAMA-09' 
field='GAMA-12' 
field='GAMA-15' 
field='NGP' 
field='SGP' 
field='Lockman-SWIRE' 
field='SA13' 
field='SPIRE-NEP' 
field='SSDF' 
field='xFLS' 
field='XMM-13hr' 


In [2]:
from herschelhelp_internal import git_version
print("This notebook was run with herschelhelp_internal version: \n{}".format(git_version()))

This notebook was run with herschelhelp_internal version: 
0aab440 (Thu Mar 22 09:41:13 2018 +0000)


In [3]:
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))

from collections import OrderedDict
import os

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.table import Column, Table
import numpy as np

from herschelhelp_internal.flagging import  gaia_flag_column
from herschelhelp_internal.masterlist import nb_astcor_diag_plot, remove_duplicates
from herschelhelp_internal.utils import astrometric_correction, flux_to_mag

/Users/sjo/anaconda3/envs/herschelhelp_internal/lib/python3.6/site-packages/seaborn/apionly.py:6: UserWarning: As seaborn no longer sets a default style on import, the seaborn.apionly module is deprecated. It will be removed in a future version.
  warnings.warn(msg, UserWarning)


In [4]:
OUT_DIR =  os.environ.get('TMP_DIR', "../dmu16_allwise/data/")
try:
    os.makedirs(OUT_DIR)
except FileExistsError:
    pass

RA_COL = "servs_ra"
DEC_COL = "servs_dec"

In [5]:
## I - Reading in WISE astrometric catalogue

In [6]:
wise = Table.read(f"../dmu16_allwise/data/Allwise_PSF_stack_{field}.fits")
wise_coords=SkyCoord(wise['ra'], wise['dec'])

epoch = 2009

wise[:10].show_in_notebook()

idx,ra,dec,w4mag
,deg,deg,mag
0,203.4147077,37.4250645,9.378
1,203.2962176,37.4727302,9.044
2,203.296906,37.4777635,9.444
3,203.3143497,37.4686183,9.464
4,203.3123273,37.4762516,9.059
5,203.3177577,37.4771595,9.533
6,203.3318084,37.4661865,9.343
7,203.3342913,37.4731416,8.726
8,203.3454295,37.4668158,9.445


## III - Astrometry correction

We match the astrometry to the Gaia one. We limit the Gaia catalogue to sources with a g band flux between the 30th and the 70th percentile. Some quick tests show that this give the lower dispersion in the results.

In [7]:
#gaia = Table.read("./dmu17_XMM-LSS/data/GAIA_XMM-LSS.fits")
print(f"../../dmu0/dmu0_GAIA/data/GAIA_{field}.fits")
gaia = Table.read(f"../dmu0/dmu0_GAIA/data/GAIA_{field}.fits")
gaia_coords = SkyCoord(gaia['ra'], gaia['dec'])

../../dmu0/dmu0_GAIA/data/GAIA_XMM-13hr.fits


FileNotFoundError: [Errno 2] No such file or directory: '../dmu0/dmu0_GAIA/data/GAIA_XMM-13hr.fits'

In [ ]:
nb_astcor_diag_plot(wise_coords.ra, wise_coords.dec, 
                    gaia_coords.ra, gaia_coords.dec, near_ra0=True)

In [ ]:
delta_ra, delta_dec =  astrometric_correction(
    wise_coords,
    gaia_coords, near_ra0=True
)

print("RA correction: {}".format(delta_ra))
print("Dec correction: {}".format(delta_dec))

In [ ]:
print( wise["ra"])
print(delta_ra.to(u.deg))

In [ ]:
#wise["ra"] +=  delta_ra.to(u.deg)
wise["ra"] = wise["ra"]+ delta_ra.to(u.deg)
wise["dec"] = wise["dec"]+ delta_dec.to(u.deg)

In [ ]:
nb_astcor_diag_plot(wise["ra"], wise["dec"], 
                    gaia_coords.ra, gaia_coords.dec, near_ra0=True)

## V - Saving to disk

In [ ]:
wise.write(f"../dmu16_allwise/data/Allwise_PSF_stack_GAIA_{field}.fits", overwrite=True)